In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
# prompt: whats the pip install for models.model import


!pip install models


  Using cached models-0.9.3.tar.gz (16 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-uurxmezn/models_f858f5b19ff34358a3a1c37f500d066c/setup.py", line 25, in <module>
          import models
        File "/tmp/pip-install-uurxmezn/models_f858f5b19ff34358a3a1c37f500d066c/models/__init__.py", line 23, in <module>
          from base import *
      ModuleNotFoundError: No module named 'base'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pi

In [9]:
!git clone https://github.com/NilotpalMaitra/UDA_thermal


Cloning into 'UDA_thermal'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 65 (delta 32), reused 25 (delta 13), pack-reused 0
Unpacking objects: 100% (65/65), 5.72 MiB | 11.42 MiB/s, done.


In [10]:
import shutil
import os

# Define the source and destination paths
src_path = 'UDA_thermal/models'
dest_path = 'models'

# Move the 'models' folder to the desired location
shutil.move(src_path, dest_path)

# Optionally, remove the rest of the cloned repository
shutil.rmtree('UDA_thermal')


In [11]:
import os
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
#from dataset.data_loader import GetLoader
from torchvision import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import torch

import torchvision.transforms as T
import torchvision
device_gpu=0




def test_(dataset_name, epoch):
    assert dataset_name in ['source', 'target']

    model_root = os.path.join("/kaggle/working/UDA_thermal/UDA_thermal", 'models')
   

    cuda = True
    cudnn.benchmark = True
    batch_size = 32
    image_size = 224
    alpha = 0
    # Digits and Alpha
    #'''
    img_rgb_resized = np.load("/kaggle/input/dataset/X_rgb_224.npy")
    labels_rgb = np.load("/kaggle/input/dataset/Y_rgb_224.npy")
    img_th_rot = np.load("/kaggle/input/dataset/X_th_224-001.npy")
    labels_th = np.load("/kaggle/input/dataset/Y_th_224.npy")


    img_rgb_train, img_rgb_test, labels_rgb_train, labels_rgb_test = train_test_split(img_rgb_resized, labels_rgb, test_size=0.1, random_state=42,stratify=labels_rgb)

    img_th_train, img_th_test, labels_th_train, labels_th_test = train_test_split(img_th_rot, labels_th, test_size=0.6, random_state=42,stratify=labels_th)
    #'''
    
    '''
    img_rgb_train = np.load('/raid/ai21resch11003/DA_HG/dataset_244/train_source.npy')
    img_rgb_test = np.load('/raid/ai21resch11003/DA_HG/dataset_244/test_source.npy')
    labels_rgb_train = np.load('/raid/ai21resch11003/DA_HG/dataset_244/train_source_labels.npy')
    labels_rgb_test = np.load('/raid/ai21resch11003/DA_HG/dataset_244/test_source_labels.npy')

    img_th_train = np.load('/raid/ai21resch11003/DA_HG/dataset_244/train_target.npy')
    img_th_test = np.load('/raid/ai21resch11003/DA_HG/dataset_244/test_target.npy')
    labels_th_train = np.load('/raid/ai21resch11003/DA_HG/dataset_244/train_target_labels.npy')
    labels_th_test = np.load('/raid/ai21resch11003/DA_HG/dataset_244/test_target_labels.npy')
    #'''
  


   # '''
    train = torch.utils.data.TensorDataset(torch.from_numpy(img_rgb_train), torch.from_numpy(labels_rgb_train))
    test = torch.utils.data.TensorDataset(torch.from_numpy(img_rgb_test), torch.from_numpy(labels_rgb_test))
    train_dataloader_source = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    test_dataloader_source = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

    train_th = torch.utils.data.TensorDataset(torch.from_numpy(img_th_train), torch.from_numpy(labels_th_train))
    test_th = torch.utils.data.TensorDataset(torch.from_numpy(img_th_test), torch.from_numpy(labels_th_test))
    train_dataloader_target = torch.utils.data.DataLoader(train_th, batch_size=batch_size, shuffle=True)
    test_dataloader_target = torch.utils.data.DataLoader(test_th, batch_size=batch_size, shuffle=True)

    if dataset_name == 'source':
      dataloader= test_dataloader_source
    elif dataset_name == 'target':
      dataloader = test_dataloader_target

    """ training """

    my_net = torch.load(os.path.join(
        model_root, 'mnist_mnistm_model1_epoch_' + str(epoch) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda(device_gpu)

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = data_target_iter.next()
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda(device_gpu)
            t_label = t_label.cuda(device_gpu)
            input_img = input_img.cuda(device_gpu)
            class_label = class_label.cuda(device_gpu)

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        class_output, _ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total
    print ('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))
    return accu
    


In [12]:
# Call the test_ function with desired parameters
# Example: test the model on the 'source' dataset at epoch 10
accuracy = test_('source', 10)
print(f'Accuracy: {accuracy}')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/UDA_thermal/UDA_thermal/models/mnist_mnistm_model1_epoch_10.pth'